*parcours en largeur - M. Liedloff, janvier 2021*

# Consignes

- sur Celene, téléchargez les fichiers indiqués
- ouvrir un `terminal`, se déplacer dans le dossier où les fichiers ont été téléchargé (commande `cd`)
- tapez dans le terminal la commande : `jupyter notebook`
- ouvrir le fichier .ipynb

# Manipulation de graphes et parcours en largeur

Nous utilisons la syntaxe suivante, à base de dictionnaires et de listes, pour définir un graphe en Python : 

In [2]:
G = {
    "a": ["b"],
    "b": ["a", "c", "d"],
    "c": ["b", "d"],
    "d": ["b", "c", "e"],
    "e": ["d"]
}

Ainsi, une fonction qui parcourt toutes les arêtes d'un graphe s'écrit simplement: 

In [3]:
def AfficherAretes(G):
    for u in G:
        for v in G[u]:
            print("Arête {%s,%s}" % (u,v))

In [4]:
AfficherAretes(G)

Arête {a,b}
Arête {b,a}
Arête {b,c}
Arête {b,d}
Arête {c,b}
Arête {c,d}
Arête {d,b}
Arête {d,c}
Arête {d,e}
Arête {e,d}


Il est également possible d'afficher un graphe en utilisant [GraphViz](https://www.graphviz.org/) ([lien vers la documentation](http://graphviz.readthedocs.io/en/stable/index.html)).
Si le code ci-dessous ne fonctionne pas sur votre machine, cela n'empêchera pas le bon déroulement de la suite du TP. Cela indique seulement que la librairie graphviz n'est pas installée.

In [6]:
from graphviz import Graph
dot = Graph()

for v in G:
    dot.node(v, "sommet "+v)

for v in G:
    for u in G[v]:
        if v<=u:
            dot.edge(v, u)
print(dot)

graph {
	a [label="sommet a"]
	b [label="sommet b"]
	c [label="sommet c"]
	d [label="sommet d"]
	e [label="sommet e"]
	a -- b
	b -- c
	b -- d
	c -- d
	d -- e
}



Il existe aussi des outils de visualiation en ligne : https://dreampuf.github.io/GraphvizOnline/

Petite astuce: les **[dictionnaires](https://docs.python.org/3.5/tutorial/datastructures.html#dictionaries)** sont très utiles pour définir des états (ou d'autres choses). On peut définir une variable etats et s'en servir plus tard de la façon suivante :

```
etats = {}
...
etats[u] = "non atteint"
...
if etats[u]=="non atteint":
etats[u] = "atteint"
```

On peut aussi utiliser des **[ensembles](https://docs.python.org/3.5/tutorial/datastructures.html#sets)** :
```
sommetsAtteints = set()
...
sommetsAtteints.add(u)
...
if u not in sommetsAtteints:
# do something
```

Une **file FIFO** peut être implémentée en utilisant une *liste*.
On y ajoute un élément avec la méthode `append`. Un élément est extrait avec la méthode `pop`. Voici un exemple pour manipuler une liste comme une file FIFO. Prenez le temps de lire ce code et de regarder le résultat produit.

In [7]:
def estvide(F):
    """ cette fonction retourne True si la liste est vide, sinon elle retourne False"""
    if F:
        return False
    else:
        return True

F = ["élément A"] # pour initialiser une file vide : F = []
F.append("élément B") # ajoute un élément
print(F)
e = F.pop(0) # défile un élément
print(e)
F.append("élément C")
print(F)

print("longueur de la liste", len(F))

# tant que la file n'est pas vide, on défile un élément et on l'affiche :
while F:
    print(F.pop(0))

print(estvide(F))

['élément A', 'élément B']
élément A
['élément B', 'élément C']
longueur de la liste 2
élément B
élément C
True


# Parcours en largeur

## question 1
Écrire une fonction `ParcoursLargeur(G,s)` de parcours en largeur, la mettre au point, et la tester sur le graphe `G` précédent (ou un autre graphe de votre choix). Cette fonction attend en entrée un graphe `G` et un sommet `s` à partir duquel le graphe est exploré.

Pour le moment, vous n'avez pas besoin de gérer la distance `d` et le père `pere` de chaque sommet.

![algorithme du parcours en largeur](http://www.univ-orleans.fr/lifo/Members/Mathieu.Liedloff/temp/ACSD/TP1/parcourslargeur.png)

In [14]:
def ParcoursLargeur(G,s):
    etats = {}
    distance = {}
    pere = {}
    for i in G.keys() :
        etats[i] = 'non atteint'
        distance[i] = -1
        pere[i] = None
    
    etats[s] = 'atteint'
    distance[s] = 0
    pere[s] = None

    F = [s]

    while F :
        head = F[0]
        for i in G[head] :
            if etats[i] == 'non atteint':
                etats[i] = 'atteint'
                distance[i] = distance[head] + 1
                pere[i] = head
                F.append(i)
        F.pop(0)
        etats[head] = 'traité'
    return (etats,distance,pere)

In [15]:
ParcoursLargeur(G,'a')

({'a': 'traité', 'b': 'traité', 'c': 'traité', 'd': 'traité', 'e': 'traité'},
 {'a': 0, 'b': 1, 'c': 2, 'd': 2, 'e': 3},
 {'a': None, 'b': 'a', 'c': 'b', 'd': 'b', 'e': 'd'})

## question 2
Un cookie a été caché sur l'un des sommets d'un graphe. Vous ne connaissez pas ce graphe (il est stocké sur un serveur), mais vous disposez des fonctions suivantes :
- **`getAVertex()`** retourne un sommet du graphe à partir duquel vous commencez l'exploration du graphe.
- **`getNeighbors(u)`** retourne la liste des voisins d'un sommet `u`.
- **`Cookie(u)`** retourne True si et seulement si un cookie se trouve sur le sommet `u`.

Pour vous connectez au serveur contenant le graphe, nous importons quelques fonctions puis nous utilisons la fonction connectSocket: 

In [2]:
from utils import connectSocket, closeSocket, sendcmd, recvcmd
from utils import getNeighbors, getAVertex, Cookie, CookieDistance

#connectSocket("192.168.47.162", 6667)
#connectSocket("uomobile.univ-orleans.fr", 6667)
connectSocket("distrigraphes.info-orleans.fr", 8080)

Attention: le graphe change à chaque connexion, sinon ce serait trop facile ! Il faut donc que votre algorithme de parcours soit vraiment au point.

Voici un petit exemple d'utilisation de ces fonctions:

In [ ]:
sommetInitial = getAVertex()
print("Le sommet initial est", sommetInitial)

for v in getNeighbors(sommetInitial):
    print("Le sommet",sommetInitial, "est voisin avec", v)

print("Le sommet ",sommetInitial, "contient-il un cookie ?", Cookie(sommetInitial))  

>>> GetStartingVertex 
<<< StartingVertex AA077
Le sommet initial est AA077
>>> GetNeighbor AA077
<<< Neighborhood YM182#WM265#QT398
Le sommet AA077 est voisin avec YM182
Le sommet AA077 est voisin avec WM265
Le sommet AA077 est voisin avec QT398
>>> Cookie? AA077
<<< Cookie No
Le sommet  AA077 contient-il un cookie ? False


Les lignes avec des chevrons `>>>` et `<<<` sont simplement les messages échangés entre votre machine et le serveur. Vous pouvez les ignorer ...

Il faut maintenant adaptez votre algorithme de parcours en largeur pour écrire une fonction **`ParcoursLargeurNetwork()`** qui parcourt le graphe du serveur et affiche le nom du sommet contenant le cookie. 
**Saurez-vous trouver le cookie dans le graphe ?**

In [8]:
def ParcoursLargeurNetwork():
    s = getAVertex()

    F = [s]

    i = 0 

    while F :
        head = F[i]
        i+=1
        if Cookie(head) == True : return head
        for element in getNeighbors(head) :
            if element not in F : F.append(element)

        print(F, i)

In [10]:
coo = ParcoursLargeurNetwork()
print("Le cookie est sur le sommet", coo)

>>> GetStartingVertex 
<<< StartingVertex AA598
>>> Cookie? AA598
<<< Cookie No
>>> GetNeighbor AA598
<<< Neighborhood HG768#FM711#BD817#FX998
['AA598', 'HG768', 'FM711', 'BD817', 'FX998'] 1
>>> Cookie? HG768
<<< Cookie No
>>> GetNeighbor HG768
<<< Neighborhood NS243#AA598#BD817#LG667
['AA598', 'HG768', 'FM711', 'BD817', 'FX998', 'NS243', 'LG667'] 2
>>> Cookie? FM711
<<< Cookie No
>>> GetNeighbor FM711
<<< Neighborhood AA598#NZ056#UZ312#AE494#YR091
['AA598', 'HG768', 'FM711', 'BD817', 'FX998', 'NS243', 'LG667', 'NZ056', 'UZ312', 'AE494', 'YR091'] 3
>>> Cookie? BD817
<<< Cookie No
>>> GetNeighbor BD817
<<< Neighborhood TE502#EY415#HG768#AA598#GH433#RD298#EY814#MM010
['AA598', 'HG768', 'FM711', 'BD817', 'FX998', 'NS243', 'LG667', 'NZ056', 'UZ312', 'AE494', 'YR091', 'TE502', 'EY415', 'GH433', 'RD298', 'EY814', 'MM010'] 4
>>> Cookie? FX998
<<< Cookie No
>>> GetNeighbor FX998
<<< Neighborhood TR758#EE437#AA598#JA163#SU003#AX535#CR050
['AA598', 'HG768', 'FM711', 'BD817', 'FX998', 'NS243', 'L

## question 3

Déterminez la distance entre le sommet retourné par `getAVertex()` et le sommet contenant le cookie. Pour vérifier votre résultat, utilisez la fonction **`CookieDistance(r)`** qui attend en paramètre la distance `r` que vous avez calculée et retourne `True` si et seulement si votre résultat est correct. 

In [11]:


def ParcoursLargeurNetwork():
    s = getAVertex()

    explored = []
    exploring = [s]
    toExplore = []
    distance = 0

    while exploring :
        for vertex in exploring :
            if Cookie(vertex) == True : return vertex, distance
            for verticies in getNeighbors(vertex) :
                if verticies not in explored and verticies not in exploring and verticies not in toExplore : toExplore.append(verticies)

        explored += exploring + []
        exploring = toExplore + []
        distance += 1
        
(coo, dist) = ParcoursLargeurNetwork()

print("Le cookie est sur le sommet", coo, Cookie(coo) )
print("Le cookie est à distance", dist, CookieDistance(dist))

>>> GetStartingVertex 
<<< StartingVertex AB878
0
>>> Cookie? AB878
<<< Cookie No
>>> GetNeighbor AB878
<<< Neighborhood KJ592#JJ639#XX955#BB838#LP223#VI445#EB746#RY475#VR728
1
>>> Cookie? KJ592
<<< Cookie No
>>> GetNeighbor KJ592
<<< Neighborhood XL582#AB878#LC606#MQ163
>>> Cookie? JJ639
<<< Cookie No
>>> GetNeighbor JJ639
<<< Neighborhood AB878#LY508#HQ704#KR303#PI011#IB460
>>> Cookie? XX955
<<< Cookie No
>>> GetNeighbor XX955
<<< Neighborhood JF442#CT691#AB878#GM772#XL582#GG881
>>> Cookie? BB838
<<< Cookie No
>>> GetNeighbor BB838
<<< Neighborhood HW080#YL746#UG702#GH317#CA418#AB878#NA056#JF251#PZ430#QA352
>>> Cookie? LP223
<<< Cookie No
>>> GetNeighbor LP223
<<< Neighborhood IA188#PZ430#AB878#WG749#XI947#RY016
>>> Cookie? VI445
<<< Cookie No
>>> GetNeighbor VI445
<<< Neighborhood IZ774#IO482#NP607#PB609#AB878#AV857
>>> Cookie? EB746
<<< Cookie No
>>> GetNeighbor EB746
<<< Neighborhood PH333#AB878
>>> Cookie? RY475
<<< Cookie No
>>> GetNeighbor RY475
<<< Neighborhood OX829#KB930#XL5

## question 4
Construire un plus court chemin du sommet de départ retourné par `getAVertex()` vers le sommet contenant le cookie.

In [12]:
def ParcoursLargeurNetwork():
    s = getAVertex()

    pere = {}

    explored = []
    exploring = [s]
    toExplore = []

    pere[s] = None
    running = True
    distance = 0

    while exploring and running :
        for vertex in exploring :
            if Cookie(vertex) == True :
                cookieVertex = vertex
                running = False
                break
            for verticies in getNeighbors(vertex) :
                if verticies not in pere.keys() :
                    toExplore.append(verticies)
                    pere[verticies] = vertex

        explored += exploring + []
        exploring = toExplore + []
        distance += 1

    chain = cookieVertex
    
    while True :
        if pere[cookieVertex] == None : return chain
        cookieVertex = pere[cookieVertex]
        chain = cookieVertex + " -> " + chain

print(ParcoursLargeurNetwork())

>>> GetStartingVertex 
<<< StartingVertex AB878
>>> Cookie? AB878
<<< Cookie No
>>> GetNeighbor AB878
<<< Neighborhood KJ592#JJ639#XX955#BB838#LP223#VI445#EB746#RY475#VR728
>>> Cookie? KJ592
<<< Cookie No
>>> GetNeighbor KJ592
<<< Neighborhood XL582#AB878#LC606#MQ163
>>> Cookie? JJ639
<<< Cookie No
>>> GetNeighbor JJ639
<<< Neighborhood AB878#LY508#HQ704#KR303#PI011#IB460
>>> Cookie? XX955
<<< Cookie No
>>> GetNeighbor XX955
<<< Neighborhood JF442#CT691#AB878#GM772#XL582#GG881
>>> Cookie? BB838
<<< Cookie No
>>> GetNeighbor BB838
<<< Neighborhood HW080#YL746#UG702#GH317#CA418#AB878#NA056#JF251#PZ430#QA352
>>> Cookie? LP223
<<< Cookie No
>>> GetNeighbor LP223
<<< Neighborhood IA188#PZ430#AB878#WG749#XI947#RY016
>>> Cookie? VI445
<<< Cookie No
>>> GetNeighbor VI445
<<< Neighborhood IZ774#IO482#NP607#PB609#AB878#AV857
>>> Cookie? EB746
<<< Cookie No
>>> GetNeighbor EB746
<<< Neighborhood PH333#AB878
>>> Cookie? RY475
<<< Cookie No
>>> GetNeighbor RY475
<<< Neighborhood OX829#KB930#XL582#R